## How to use Yelp API v3 to get Business Data

* In my code, I majorly created:
  * How to search for business using its name, address1, city, state and country, and get business id
  * How to get more business details with business id
  * How to get reviews for the business
    * But Yelp give you at most 3 reviews, per business

* First of all, you have to create an app so that you can get API key
  * https://www.yelp.com/developers/documentation/v3/authentication
  * Need to use a browser that won't block javascript, otherwise you may not get API key
  * Once you got the API key, Yelp will also tell you your daily limitation
* URLs for getting Yelp business data: https://www.yelp.com/developers/documentation/v3/get_started

In [2]:
import requests
from urllib import quote
from pprint import pprint

API_HOST = 'https://api.yelp.com'
BUSINESS_PATH = '/v3/businesses/'
BUSINESS_MATCH_PATH = '/v3/businesses/matches'

In [3]:
api_key = '[YOU API KEY]'

In [4]:
def request(host, path, api_key, url_params=None):
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }

    print(u'Querying {0} ...'.format(url))

    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()

def get_business(api_key, business_id):
    """
    Get business info through business id
    """
    business_path = BUSINESS_PATH + business_id

    return request(API_HOST, business_path, api_key)

def get_business_reviews(api_key, business_id):
    """
    Get business info through business id
    """
    business_path = BUSINESS_PATH + business_id + '/reviews'

    return request(API_HOST, business_path, api_key)

def match_business(api_key, params):
    """
    Get business info through business name
    """
    business_path = BUSINESS_MATCH_PATH

    return request(API_HOST, business_path, api_key, params)

### match a business through REQUIRED parameters

* name
* address1
* city
* state
* country

In [5]:
params = {
    'name': 'bottega-louie-los-angeles',
    'address1': '700 S Grand Ave',
    'city': 'Los Angeles',
    'state': 'CA',
    'country': 'US'
}

match_result = match_business(api_key, params)
match_result

Querying https://api.yelp.com/v3/businesses/matches ...


{u'businesses': [{u'alias': u'bottega-louie-los-angeles',
   u'coordinates': {u'latitude': 34.0469300995766,
    u'longitude': -118.256601457672},
   u'display_phone': u'(213) 802-1470',
   u'id': u'TkFEKhsCixPWlShULKvMdQ',
   u'location': {u'address1': u'700 S Grand Ave',
    u'address2': None,
    u'address3': u'',
    u'city': u'Los Angeles',
    u'country': u'US',
    u'display_address': [u'700 S Grand Ave', u'Los Angeles, CA 90017'],
    u'state': u'CA',
    u'zip_code': u'90017'},
   u'name': u'Bottega Louie',
   u'phone': u'+12138021470'}]}

### Get More Business Info

Such as:
* Opening Hours
* Rating, Number of reviews
* Categories
* etc.

In [13]:
for buz in match_result['businesses']:
    buz_id = buz['id']
    buz_info = get_business(api_key, buz_id)
    pprint(buz_info)

Querying https://api.yelp.com/v3/businesses/TkFEKhsCixPWlShULKvMdQ ...
{u'alias': u'bottega-louie-los-angeles',
 u'categories': [{u'alias': u'italian', u'title': u'Italian'},
                 {u'alias': u'bakeries', u'title': u'Bakeries'},
                 {u'alias': u'breakfast_brunch',
                  u'title': u'Breakfast & Brunch'}],
 u'coordinates': {u'latitude': 34.0469300995766,
                  u'longitude': -118.256601457672},
 u'display_phone': u'(213) 802-1470',
 u'hours': [{u'hours_type': u'REGULAR',
             u'is_open_now': False,
             u'open': [{u'day': 0,
                        u'end': u'2200',
                        u'is_overnight': False,
                        u'start': u'0700'},
                       {u'day': 1,
                        u'end': u'2200',
                        u'is_overnight': False,
                        u'start': u'0700'},
                       {u'day': 2,
                        u'end': u'2200',
                        u'is_ov

### Get Reviews

At most 3 reviews per business.

In [19]:
for buz in match_result['businesses']:
    buz_id = buz['id']
    buz_reviews = get_business_reviews(api_key, buz_id)
    reviews = buz_reviews['reviews']
    for review in reviews:
        pprint(review)
        print

Querying https://api.yelp.com/v3/businesses/TkFEKhsCixPWlShULKvMdQ/reviews ...
{u'id': u'Llxk35CfF2ht9Xw-GOWNFg',
 u'rating': 5,
 u'text': u'I was quite impressed with this restaurant, I was expecting it to be so expensive and we went during lunch and it was actually very affordable. I ordered...',
 u'time_created': u'2018-10-22 21:35:34',
 u'url': u'https://www.yelp.com/biz/bottega-louie-los-angeles?hrid=Llxk35CfF2ht9Xw-GOWNFg&adjust_creative=h4tU1P3QBFnp_8aL3Z34gQ&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=h4tU1P3QBFnp_8aL3Z34gQ',
 u'user': {u'id': u'g05jfdQIwe_ANt75NprQkw',
           u'image_url': u'https://s3-media4.fl.yelpcdn.com/photo/dNVhUtWKaReeVXwhYIW9Iw/o.jpg',
           u'name': u'Karine L.',
           u'profile_url': u'https://www.yelp.com/user_details?userid=g05jfdQIwe_ANt75NprQkw'}}

{u'id': u'1H6D3ZvWVszpgmeSTC9nPg',
 u'rating': 5,
 u'text': u'Love this restaurant for Date night , Girls Lunch\nDrinks  \nFood looks\n& Taste  great .',
 u'tim

## Search Business Through Query

* In Yelp v3 API, you eed to use GraphQL to do some simple query, otherwise business search needs business id.

* How to use Yelp Graphql: https://github.com/gfairchild/yelpapi/blob/master/yelpapi/yelpapi.py
* Example Queries: https://github.com/gfairchild/yelpapi/blob/master/examples/examples.py

In [18]:
from yelpapi import YelpAPI
yelp_api = YelpAPI(api_key)
search_results = yelp_api.search_query(term='high tea', location='los angelas', sort_by='rating', limit=1)
search_results

{u'businesses': [{u'alias': u'cafe-giverny-los-angeles',
   u'categories': [{u'alias': u'desserts', u'title': u'Desserts'},
    {u'alias': u'coffee', u'title': u'Coffee & Tea'},
    {u'alias': u'sandwiches', u'title': u'Sandwiches'}],
   u'coordinates': {u'latitude': 34.0695915222168,
    u'longitude': -118.307540893555},
   u'display_phone': u'(213) 637-0204',
   u'distance': 1551.1451863805846,
   u'id': u'K2xIhzZAGfK4seqfkjRfCA',
   u'image_url': u'https://s3-media3.fl.yelpcdn.com/bphoto/M2nyIeE21PtnfwPpP23csA/o.jpg',
   u'is_closed': False,
   u'location': {u'address1': u'252 S Oxford Ave',
    u'address2': u'',
    u'address3': u'',
    u'city': u'Los Angeles',
    u'country': u'US',
    u'display_address': [u'252 S Oxford Ave', u'Los Angeles, CA 90004'],
    u'state': u'CA',
    u'zip_code': u'90004'},
   u'name': u'Cafe Giverny',
   u'phone': u'+12136370204',
   u'price': u'$$',
   u'rating': 4.5,
   u'review_count': 131,
   u'transactions': [],
   u'url': u'https://www.yelp.com